In [ ]:
import pandas as pd
import numpy as np
from plotly.offline import iplot, init_notebook_mode 
# Trasforming df into cufflinks for iPlot 
import cufflinks 
cufflinks.go_offline(connected= True) 
init_notebook_mode(connected= True)

import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:
coviddf = pd.read_csv('https://raw.githubusercontent.com/CarmineCrown/COVID-19/master/covid_19_datasets/covid_19_globaldata/covid_19_world.csv')

In [ ]:
coviddf

# Data Manipulation
## Data time Formatting

In [ ]:
coviddf['ObservationDate'] = pd.to_datetime(coviddf['ObservationDate'])

In [ ]:
coviddf.head()

In [ ]:
covid_datewise = coviddf.groupby(['ObservationDate']).agg({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum'})

In [ ]:
covid_datewise.head()

In [ ]:
covid_datewise['Active'] = covid_datewise['Confirmed'] - covid_datewise['Deaths'] - covid_datewise['Recovered']

In [ ]:
covid_datewise.head()

iPlot

In [ ]:
covid_datewise.iplot()

SeaBorn

In [ ]:
sns.set_style('darkgrid') 
plt.figure(figsize=(20,20)) 
sns.barplot(x = covid_datewise.index.date, y = covid_datewise['Confirmed'], palette='YlOrRd') 
plt.xticks(rotation = 90) 
plt.show()

## Mortality Rate

In [ ]:
covid_datewise['Mortality Rate'] = (covid_datewise['Deaths']) / (covid_datewise['Confirmed'] ) * 100

iPlot

In [ ]:
covid_datewise['Mortality Rate'].iplot()

In [ ]:
covid_datewise['Days'] = covid_datewise.index - covid_datewise.index[0]

## Linear Regression

In [ ]:
covid_datewise 

In [ ]:
covid_datewise ['Days'] = covid_datewise['Days'].dt.days

In [ ]:
covid_datewise

In [ ]:
# First 90 % rows for training 
train_ml = covid_datewise.iloc [: int(covid_datewise.shape[0]*.90)]

In [ ]:
train_ml

In [ ]:
test_ml = covid_datewise.iloc [int(covid_datewise.shape[0]*.90):]

In [ ]:
test_ml

In [ ]:
test_ml.shape

In [ ]:
train_ml.shape

### Linear Regression Algorithm

In [ ]:
from sklearn.linear_model import LinearRegression 
lin_reg = LinearRegression(normalize= True)

In [ ]:
X = np.array(train_ml['Days']).reshape(-1,1)
Y = np.array(train_ml['Confirmed']).reshape(-1,1)

In [ ]:
print(X)
print(Y)

    Training of Data 

In [ ]:

## Predictions
lin_reg.fit(X, Y)
predicted_value = lin_reg.predict(np.array(test_ml['Days']).reshape(-1,1))

    Predictions

In [ ]:
 predicted_value = lin_reg.predict(np.array(test_ml['Days']).reshape(-1,1))

In [ ]:
predicted_value

In [ ]:
predicted_value

# Implement Regression

In [ ]:
predicted_value_all = lin_reg.predict(np.array(covid_datewise['Days']).reshape(-1,1))

In [ ]:
plt.figure(figsize=(25,10)) 
plt.plot(covid_datewise['Confirmed'], label = 'Confirmed') 
plt.plot(covid_datewise.index, predicted_value_all, label = 'Predicted Confirmed Cases') 
plt.legend()
plt.show()

# FBProphet: ARIMA

                                ---- Could not complete because could not detect FBProphet ----

# RNN Recurrent Neural Network

## RNN - LSTM - Times Series Forecasting 
#### Neural Network with Memory 
#### Output will depend upon features (X) and past memory (logic gates that stores part memory)​

In [ ]:
# Sequential arrangement of Neural Network
from keras.models import Sequential 
# LSTM - Part of RNN 
from keras.layers import LSTM, Dense 
from keras.layers import Dropout 
from sklearn.preprocessing import MinMaxScaler 
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
data = coviddf[coviddf['Country/Region'] == 'India']

In [ ]:
data

In [ ]:
data = data.loc[:, ['ObservationDate', 'Confirmed']]

In [ ]:
data

In [ ]:
data = data.groupby('ObservationDate')[['Confirmed']].max().reset_index()

In [ ]:
data

In [ ]:
dataset = data.drop("ObservationDate", axis = 1)

In [ ]:
dataset

In [ ]:
data = np.array(dataset).reshape(-1, 1)

In [ ]:
train_data = dataset[:len(dataset)-5] 
# For validation take last 5 Values 
test_data = dataset[len(dataset)-5:]

In [ ]:
# Data Preprocessing 
# scaling large samples into equal range of (0,1) 
scaler = MinMaxScaler() 
scaler.fit(train_data) 
scaled_train_data = scaler.transform(train_data) 
scaled_test_data = scaler.transform(test_data)

In [ ]:
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=10)

In [ ]:
## Lstm Model 
lstm_model = Sequential() 
# layer - 1 : Input Layer 
# units = no of neurons , backpropogation: return_sequences, 
lstm_model.add(LSTM(units = 64, return_sequences= True, input_shape = (n_input, n_feature))) 
# Avoid overfitting of model 0.2 = 20% 
lstm_model.add(Dropout(0.2)) 
# Hidden Layer - I
lstm_model.add(LSTM(units = 64, return_sequences= True)) 
lstm_model.add(Dropout(0.2))
# Hidden Layer - II 
lstm_model.add(LSTM(units = 64)) 
lstm_model.add(Dropout(0.2)) 
# Ouput layer 
# In regression : last layer neuron = 1 
lstm_model.add(Dense(units = 1)) 
## Optimization & Loss Function 
lstm_model.compile(optimizer = 'adam', loss='mean_squared_error')